<a href="https://colab.research.google.com/github/hturbe/ECG_MAR_Model/blob/main/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECG classification using MAR coefficients
The notebook belows trains a LightGBM model on the disease of interest. There is the possibility to perform an hyperparameter search as well as specifying custom parameters for the model. In the second part of the notebook, interpretability of the model is presented using the Shap framework.

## Evironement setup
The lines below clone the github repository if the notebook is run in google colab, install requirements as well as downloand the raw data and unzip them

In [ ]:
#Re clone git to acess all scripts if run from google Colab as well as install dependence
import sys
if 'google.colab' in sys.modules:
  !git clone https://github.com/hturbe/ECG_MAR_Model.git
  !rm ECG_MAR_Model/main_notebook.ipynb
  !mv  ECG_MAR_Model/* .
  !rm -r ECG_MAR_Model

  # Install requirements: 
  !pip install -r requirements.txt

In [ ]:
# Download and unzip data
!mkdir data
!wget -O data/PhysioNetChallenge2020_Training_CPSC.zip https://sandbox.zenodo.org/record/1036220/files/PhysioNetChallenge2020_Training_CPSC.zip?download=1  
!unzip -qo data/PhysioNetChallenge2020_Training_CPSC.zip -d data/

In [ ]:
import os

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score,
                             classification_report, confusion_matrix,
                             mean_squared_error)
from sklearn.model_selection import KFold  # for K-fold cross validation
from sklearn.model_selection import cross_validate  # score evaluation
from sklearn.model_selection import train_test_split

# print the JS visualization code to the notebook
shap.initjs()

CWD = os.getcwd()



## Data Preprocessing 
The python file `__exec_data_processing.py` should be run before launching this notebook in order to generate the required data
uncomment the cell below to run the full preprocessing pipeline
It will take a bit of time to extract the signal from the petastorm file (~ 10 minutes)
The permutation test will also take 10 minutes to run

In [ ]:
#To run both the preprocessing and the permutation test, uncomment the following cell
# %run __exec_data_processing.py --operations extract_signal compute_permutation

# To run only the preprocessing  uncomment the following cell
%run  __exec_data_processing.py --operations extract_signal


## Select disease of interest
The cell below is used to select the disease of interest. Should be one of [lbbb,rbbb]

In [ ]:
disease_interest = "lbbb"

In [ ]:
signal_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
array_names = []
for i in range(len(signal_names)):
    for j in range(len(signal_names)):
            # array_names.append('X'+signal_names[j] + '-Y' + signal_names[i])
            array_names.append(signal_names[j] + '-' + signal_names[i])

## Load data

In [ ]:
path_coeff = os.path.join(CWD, 'data',"coeff_matrices")
X_normal  = np.load(os.path.join(path_coeff,'coeff_normal.npy'))
y_normal = np.zeros(X_normal.shape[0])
print("Number of ECGs without disease", X_normal.shape[0])

X_disease = np.load(os.path.join(path_coeff,f"coeff_{disease_interest}.npy"))
y_disease = np.ones(X_disease.shape[0])
print(f"Number of ECGs with {disease_interest}", X_disease.shape[0])

X_all = np.concatenate((X_normal, X_disease), axis=0)
X_all = X_all.reshape(X_all.shape[0], -1)
y_all = np.concatenate((y_normal, y_disease), axis=0)

df_all = pd.DataFrame(X_all, columns=array_names)
X_train, X_test, y_train, y_test = train_test_split(df_all, y_all, test_size=0.1, 
                                                    
                                                   )
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)

In [ ]:
#To train model with weighted loss
weights_y = y_all.shape[0]/y_all.sum()

## Train LightGBM tree model

### Train the model using user-specified parameters

In [ ]:
params = {
    "max_bin": 512,
    "learning_rate": 0.12,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "num_leaves": 1420,
    "max_depth": 4,
    "boost_from_average": True,
    # "class_weight" : {1: weights_y, 0: 1}, #To train model with weighted loss
    'is_unbalance':True,
}
metric_scoring = ["accuracy", "precision", "recall", "f1"]

clf = lgb.LGBMClassifier(
    **params
)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
kfold = KFold(n_splits=10, shuffle = True) # k=10 splits the data into 10 equal parts
cv_result = cross_validate(lgb.LGBMClassifier(**params),X_all, y_all, cv = 10, scoring = ["accuracy", "precision", "recall", "f1"])

score_mean = {x:cv_result[x].mean() for x in cv_result.keys()}
print("Cross-validation metrics: \n", os.linesep.join(f"{x}: {score_mean[x]}" for x in score_mean.keys()))


cm = confusion_matrix(y_test, pd.Series(y_pred).round() )
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             )
disp.plot() 

### Perform Hyperparameters optimisation 
Uncomment the lines in the cell below if you would like to perform hyperparameters optimisation

In [ ]:
"""
from sklearn.model_selection import GridSearchCV
params = { 'max_depth': [3,6,10,20],
           'learning_rate': [0.005,0.01, 0.05],
           "max_bin": [128,256,512],
           "num_leaves": [50,50,100],
           "num_leaves": 2000,
         "max_depth": 6,
            }
xgbr =lgb.LGBMClassifier(seed = 20, objective= "binary",
        metric= "binary_logloss",)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='accuracy', 
                   verbose=1,
                   n_jobs=4,)
clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (clf.best_score_))
"""

In [ ]:
print(classification_report(y_test, pd.Series(y_pred).round() ))

## Model Interpretability

In [ ]:
shap_values = shap.TreeExplainer(clf).shap_values(df_all)
shap.summary_plot(shap_values[1], df_all)

In [ ]:
lead_interest = "V1-aVR"

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
shap.dependence_plot(lead_interest, shap_values[1], df_all, interaction_index=None,ax = ax, show=False)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
ax.tick_params(axis='both', which='major', labelsize=16)
plt.tight_layout()
# plt.savefig(f"shap_{lead_interest}.png")